# Adding Dependeces

In [18]:
using DelimitedFiles
using CUDA
using ProgressMeter
include("../../src/forces.jl")
include("../../src/Neighbor.jl")

knn_cu (generic function with 1 method)

# Initial Conditions

In [19]:
# Running fusion of the two aggregates
# Physical Conditions
R_agg = 10

# Time model Conditions
T_f = 150000
dt = 0.1

# Neighbor Conditions
n_knn = 50
nn = 16

# Forces Conditions
R_Max = 3.4
S = 1.9
K_array = 0.05

# Coding C0nditions
n_text = 50

4

# Running Fusion

In [20]:
# # Testing all the conditions of the fusion
# for t_f in T_f
#     for r_max in R_Max
#         for s in S
#             for K in K_array
#                 # Initial Data
#                 X = Float32.(readdlm("../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu
                
#                 #Generating the path
#                 if !isdir("../../results/fusion/T_$(t_f)") mkdir("../../results/fusion/T_$(t_f)") end
#                 if !isdir("../../results/fusion/T_$(t_f)/rmax_$(r_max)_s_$(s)") mkdir("../../results/fusion/T_$(t_f)/rmax_$(r_max)_s_$(s)") end
#                 if !isdir("../../results/fusion/T_$(t_f)/rmax_$(r_max)_s_$(s)/k_$(K)") mkdir("../../results/fusion/T_$(t_f)/rmax_$(r_max)_s_$(s)/k_$(K)")  end
                
#                 # Calculating all above
#                 if size(readdir("../../results/fusion/T_$(T_f)/rmax_$(r_max)_s_$(s)/k_$(K)"))[1] < 2
#                     println("Calculating T_Final=$(t_f) | R_Max = $(r_max) | s = $(s) | k = $(K)")
#                     fusion("../../results/fusion/T_$(T_f)/rmax_$(r_max)_s_$(s)/k_$(K)",n_text,t_f, r_max, s, K)
#                 else
#                     println("This is already calculated")
#                 end
#             end
#         end
#     end
# end

In [21]:
X = Float32.(readdlm("../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu
println("Calculating T_Final=$(T_f) | R_Max = $(R_Max) | s = $(S) | k = $(K_array)")
fusion("",n_text,T_f, R_Max, S, K_array)

Calculating T_Final=150000 | R_Max = 3.4 | s = 1.9 | k = 0.5


Progress:  38%|█████████▌               |  ETA: 0:09:37m41m

LoadError: InterruptException: